# Mach3 tools file format
From https://www.cnczone.com/forums/mach-software-artsoft-software-/142166-mach3-tools3-dat-tool-table-binary-file.html

The file starts with 255 tool records (#0-254), Each record is variable length.
The end of the file is padded with 255 bytes of 0x01

## Tool record file format
* first byte is length of name (N)
* next N bytes is name
* 8 bytes ignored
* 8 bytes diameter double
* 26 bytes ignored
* 8 bytes diameter wear double
* 8 bytes height double
* 8 bytes height wear double

## Improvements
- Read the fill as one bytearray, close the fill and process, same for write.



In [2]:
# class to handle MACH3 tool table binary file

import os.path 
import struct
import sys

debug = False
d_max = 5

class Mach3ToolsTable:

    def __init__(self):
        self.tools = []
        self.nbMach3ToolMax = 255

    def add_tool(self, tool):
        toolCopy = Tool()
        toolCopy.Number = tool.Number
        toolCopy.Name = tool.Name
        toolCopy.Diameter = tool.Diameter
        toolCopy.Height = tool.Height
        toolCopy.DiameterWear = tool.DiameterWear
        toolCopy.HeightWear = tool.HeightWear
        
        self.tools.append(toolCopy)
                 
    def get_tool(self, number):
        return self.tools[number]
    
    def get_printLineFormat(self):
        sep = '__________________________________________________________________________________'
        #     'Number  Name                   Diameter     Height  DiameterWear  HeightWear'
        #      001  #1_____  tool_number_1             6.000   -100.000         0.000       0.000
        #      123  #123456--12345678901234567890--123456789--123456789--123456789012--1234567890
        tool_list_style_format = '{:>3}  #{:<6}  {:<20}  {:>9}  {:>9}  {:>12}  {:>10}'
        return tool_list_style_format, sep
    
    def print(self, nFirst=None):
        toolformat, sep = self.get_printLineFormat()
        print(sep)
        print(toolformat.format('Num', 'Number', 'Name', 'Diameter', 'Height', 'DiameterWear', 'HeightWear'))
        print(sep)
        i = 1
        if nFirst is None:
            for tool in self.tools:
                tool.print(i, toolformat)
                i += 1
        else:
            for tool in self.tools[0:nFirst]:
                tool.print(i, toolformat)
                i += 1
        print(sep)
        
    def read(self, filename):
        # Read a tool from the MACH3 binary file
        if not(os.path.isfile(filename)):
            print('###ERROR: No file', filename, 'found!')
            return False
            
        aTool = Tool(1, 'Empty', 0.0, 0.0, 0.0, 0.0)
        file = open(filename,"rb")
        for i in range(0, self.nbMach3ToolMax, 1):
            aTool.Number = i+1

            NameLength = int(ord(file.read(1)))
            if (debug & (i<d_max)): print('NameLength:', type(NameLength), NameLength)
            buf = file.read(NameLength)
            if (debug & (i<d_max)): print('buf:', type(buf), buf)
            aTool.Name = buf.decode("utf-8")    
            if (debug & (i<d_max)): print('aTool.Name:', type(aTool.Name), aTool.Name)

            dummy = file.read(8)

            buf = file.read(8)
            if (debug & (i<d_max)): print('buf:', type(buf), buf)
            aTool.Diameter = struct.unpack('d', buf)[0]
            if (debug & (i<d_max)): print('aTool.Diameter:', type(aTool.Diameter), aTool.Diameter)

            dummy = file.read(26)

            aTool.Height = struct.unpack('d', file.read(8))[0]
            if (debug & (i<d_max)): print('aTool.Height:', type(aTool.Height), aTool.Height)

            aTool.DiameterWear = struct.unpack('d', file.read(8))[0]
            aTool.HeightWear = struct.unpack('d', file.read(8))[0]
            
            if (debug & (i<d_max)): print('aTool:', type(aTool), aTool)
            self.add_tool(aTool)
    
            if (debug & (i<d_max)):
                if (aTool.Name != 'Empty'): aTool.print(0, self.get_printLineFormat())
                print()
        
        file.close()
        return True
        
    def write(self, filename):
        _debug = True
        
        if os.path.isfile(filename):
            if _debug:
                print('###ERROR: File', filename, 'already exist, it will be deleted!')
                os.remove(filename)
            else:
                print('###ERROR: File', filename, 'already exist, it should be deleted!')
                sys.exit("###ERROR: File already exist")

        if (len(self.tools) != self.nbMach3ToolMax):
            print('###ERROR: Not the correct number of tools in Tool List!')
            print('Number of tools found:', len(self.tools), 'of', self.nbMach3ToolMax)
            sys.exit("###ERROR: Not the correct number of tools")

        paddingEOF = bytearray(bytes(255))
        if debug: print('paddingEOF:', type(paddingEOF), len(paddingEOF), paddingEOF)
            
        file = open(filename,"wb")
        
        for aTool in self.tools:
            b = bytearray(b'\x00')
            b[0] = len(aTool.Name)
            file.write(b)

            buf = str.encode(aTool.Name)
            #s = struct.pack('s', buf)
            file.write(buf)

            file.write(bytes(8))

            buf = aTool.Diameter
            s = struct.pack('d', buf)
            file.write(s)

            file.write(bytes(26))

            buf = aTool.Height
            s = struct.pack('d', buf)
            file.write(s)

            buf = aTool.DiameterWear
            s = struct.pack('d', buf)
            file.write(s)

            buf = aTool.HeightWear
            s = struct.pack('d', buf)
            file.write(s)

        file.write(paddingEOF)
        file.close()
        return True
                 


In [3]:
class Tool:
    def __init__(self, 
            Number=None, Name='Empty', 
            Diameter=0.0, Height=0.0, DiameterWear=0.0, HeightWear=0.0):
        self.Number = Number
        self.Name = Name
        self.Diameter = Diameter
        self.Height = Height
        self.DiameterWear = DiameterWear
        self.HeightWear = HeightWear
    
    def print(self, num, toolformat):
        print(toolformat.format(num, self.Number, self.Name, self.Diameter, self.Height, self.DiameterWear, self.HeightWear))
        
        

In [4]:
# Sample of use for class Mach3ToolsTable

mach3_filename_in  = 'tools3_sample.dat'
mach3_filename_out = 'tools3.dat'

mach3_Tools = Mach3ToolsTable()
if mach3_Tools.read(mach3_filename_in):

    # do something on mach3_Tools.tools if needed
    pass

    # Print the first 10 tools
    mach3_Tools.print(10)

    if mach3_Tools.write(mach3_filename_out):
        # check both files are the same (if no modification applied)
        print()
        print('Table should be same as before or as modified in the above code.')
        mach3_Tools_bis = Mach3ToolsTable()
        if mach3_Tools_bis.read(mach3_filename_in):
            mach3_Tools_bis.print(10)
    else:
        print('### ERROR cannot write output file', mach3_filename_out)
        sys.exit("### ERROR: File already exist")

else:
    print('### ERROR cannot read input file', mach3_filename_in)
    sys.exit("### ERROR: File does not exist")
    


__________________________________________________________________________________
Num  #Number  Name                   Diameter     Height  DiameterWear  HeightWear
__________________________________________________________________________________
  1  #1       Empty                       0.0        0.0           0.0         0.0
  2  #2       TOOL1                       1.0        1.5          10.0        10.5
  3  #3       TOOL2                       2.0        2.5          20.0        20.5
  4  #4       TOOL3                       3.0        3.5          30.0        30.5
  5  #5       Empty                       0.0        0.0           0.0         0.0
  6  #6       Empty                       0.0        0.0           0.0         0.0
  7  #7       Empty                       0.0        0.0           0.0         0.0
  8  #8       Empty                       0.0        0.0           0.0         0.0
  9  #9       Empty                       0.0        0.0           0.0         0.0
 10 

In [80]:
# Try read MACH3 tools list file
import os.path          # to handle files on disk (check file exist)
import struct
import sys

mach3_filename_in = 'tools3_sample.dat'

if not(os.path.isfile(mach3_filename_in)):
    print('###ERROR: No file', mach3_filename_in, 'found!')
    sys.exit("###ERROR: File does not exist")

debug = False
d_max = 5

ToolList = Mach3ToolsList()

file = open(mach3_filename_in,"rb")

nbToolMax = ToolList.nbMach3ToolMax

aToolNumber = 1
aToolNameLength = 5
aToolName = 'Empty'
aToolDiameter = 0.0
aToolHeight = 0.0
aToolDiameterWear = 0.0
aToolHeightWear = 0.0
dummy = ''

aTool = Tool(aToolNumber, aToolName, aToolDiameter, aToolHeight, aToolDiameterWear, aToolHeightWear)

for i in range(0, nbToolMax, 1):
    
    # Read a tool from the MACH3 file
    aToolNumber = i+1

    aToolNameLength = int(ord(file.read(1)))
    if (debug & (i<d_max)): print('aToolNameLength:', type(aToolNameLength), aToolNameLength)
    buf = file.read(aToolNameLength)
    if (debug & (i<d_max)): print('buf:', type(buf), buf)
    aToolName = buf.decode("utf-8")    
    if (debug & (i<d_max)): print('aToolName:', type(aToolName), aToolName)
    
    dummy = file.read(8)
    
    buf = file.read(8)
    if (debug & (i<d_max)): print('buf:', type(buf), buf)
    aToolDiameter = struct.unpack('d', buf)[0]
    if (debug & (i<d_max)): print('aToolDiameter:', type(aToolDiameter), aToolDiameter)
        
    dummy = file.read(26)
    
    aToolHeight = struct.unpack('d', file.read(8))[0]
    if (debug & (i<d_max)): print('aToolHeight:', type(aToolHeight), aToolHeight)

    aToolDiameterWear = struct.unpack('d', file.read(8))[0]
    aToolHeightWear = struct.unpack('d', file.read(8))[0]

    # Save tool to the list
    aTool.Number = aToolNumber
    aTool.Name = aToolName
    aTool.Diameter = aToolDiameter
    aTool.Height = aToolHeight
    aTool.DiameterWear = aToolDiameterWear
    aTool.HeightWear = aToolHeightWear
    if (debug & (i<d_max)): print('aTool:', type(aTool), aTool)
    ToolList.add_tool(aTool)
    
    if (debug & (i<d_max)):
        toolformat, sep = ToolList.get_printLineFormat()
        if (aToolName != 'Empty'): aTool.print(0, toolformat)
        print()

file.close()

print('Nb tools:', len(ToolList.tools))
ToolList.print(10)



Nb tools: 255
__________________________________________________________________________________
Num  #Number  Name                   Diameter     Height  DiameterWear  HeightWear
__________________________________________________________________________________
  1  #1       Empty                       0.0        0.0           0.0         0.0
  2  #2       TOOL1                       1.0        1.5          10.0        10.5
  3  #3       TOOL2                       2.0        2.5          20.0        20.5
  4  #4       TOOL3                       3.0        3.5          30.0        30.5
  5  #5       Empty                       0.0        0.0           0.0         0.0
  6  #6       Empty                       0.0        0.0           0.0         0.0
  7  #7       Empty                       0.0        0.0           0.0         0.0
  8  #8       Empty                       0.0        0.0           0.0         0.0
  9  #9       Empty                       0.0        0.0           0.0   

In [103]:
# Try to rewrite MACH3 tools list file
import os.path          # to handle files on disk (check file exist)
import struct
import sys

mach3_filename_out = 'tools3.dat'
if os.path.isfile(mach3_filename_out):
    print('###ERROR: File', mach3_filename_out, 'already exist, it will be deleted!')
    os.remove(mach3_filename_out)
    # sys.exit("###ERROR: File exist")

nbToolMax = ToolList.nbMach3ToolMax
if (len(ToolList.tools) != nbToolMax):
    print('###ERROR: Not the correct number of tools in Tool List!')
    print('Number of tools found:', len(ToolList.tools), 'of', nbToolMax)
    ToolList.print(10)
    sys.exit("###ERROR: Not the correct number of tools")

paddingEOF = bytearray(bytes(255))
print('paddingEOF:', type(paddingEOF), len(paddingEOF), paddingEOF)
for i in range(0, 255):
    paddingEOF[i] = 0x01
    
# ToolList comes from previous read cell
file = open(mach3_filename_out,"wb")

for aTool in ToolList.tools:
    # aToolNumber = aTool.Number
    aToolName = aTool.Name
    aToolNameLength = len(aToolName)
    aToolDiameter = aTool.Diameter
    aToolHeight = aTool.Height
    aToolDiameterWear = aTool.DiameterWear
    aToolHeightWear = aTool.HeightWear

    b = bytearray(b'\x00')
    b[0] = aToolNameLength
    file.write(b)

    buf = str.encode(aToolName)
    #s = struct.pack('s', buf)
    file.write(buf)

    file.write(bytes(8))
    
    buf = aToolDiameter
    s = struct.pack('d', buf)
    file.write(s)

    file.write(bytes(26))
        
    buf = aToolHeight
    s = struct.pack('d', buf)
    file.write(s)

    buf = aToolDiameterWear
    s = struct.pack('d', buf)
    file.write(s)

    buf = aToolHeightWear
    s = struct.pack('d', buf)
    file.write(s)

file.write(paddingEOF)
file.close()



###ERROR: File tools3.dat already exist, it will be deleted!
paddingEOF: <class 'bytearray'> 255 bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0